<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/STT%20JVS%20Convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
COLAB_ENVIRONMENT = True
CLEAR_LOGS = True
CLEAR_CHECKPOINTS = True

In [65]:
if COLAB_ENVIRONMENT:
  from google.colab import auth
  auth.authenticate_user()


In [66]:
if COLAB_ENVIRONMENT:
  !pip install tfds-nightly
  !pip install pydub
  !pip install tensorflow-io
else:
  !pip install --user tfds-nightly
  !pip install --user pydub
  !pip install --user tensorflow-io

In [67]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import time
import tensorflow_io as tfio

In [68]:
!git clone https://github.com/Jaidon-Smith/public_datasets.git

fatal: destination path 'public_datasets' already exists and is not an empty directory.


In [69]:
BATCH_SIZE = 8

# Setting Up TensorBoard

In [70]:
from datetime import datetime

In [71]:
logdir = 'gs://general-304503/notebook_logs/jvs_gated_convolution_stt_17_3_21/logs'
checkpoint_path = 'gs://general-304503/notebook_logs/jvs_gated_convolution_stt_17_3_21/checkpoints'

In [72]:
# Initialise Logs
if CLEAR_LOGS:
  !gsutil rm -r $logdir
if CLEAR_CHECKPOINTS:
    !gsutil rm -r $checkpoint_path

Removing gs://general-304503/notebook_logs/jvs_gated_convolution_stt_17_3_21/logs/#1615968844665942...
Removing gs://general-304503/notebook_logs/jvs_gated_convolution_stt_17_3_21/logs/events.out.tfevents.1615968844.5ba3808dffa3.58.1203.v2#1615968845892151...
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
CommandException: No URLs matched: gs://general-304503/notebook_logs/jvs_gated_convolution_stt_17_3_21/checkpoints


In [73]:
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Building the Model

In [74]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1treUWLprQkcGOXeiFPofOvtk4iPZhsJq' -O hiragana_jsut.model

--2021-03-17 08:16:36--  https://docs.google.com/uc?export=download&id=1treUWLprQkcGOXeiFPofOvtk4iPZhsJq
Resolving docs.google.com (docs.google.com)... 74.125.143.102, 74.125.143.113, 74.125.143.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.143.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mnsmrjumca6ntsrph47n9pr3erkqhgcl/1615968975000/04186398190322129029/*/1treUWLprQkcGOXeiFPofOvtk4iPZhsJq?e=download [following]
--2021-03-17 08:16:37--  https://doc-0g-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mnsmrjumca6ntsrph47n9pr3erkqhgcl/1615968975000/04186398190322129029/*/1treUWLprQkcGOXeiFPofOvtk4iPZhsJq?e=download
Resolving doc-0g-28-docs.googleusercontent.com (doc-0g-28-docs.googleusercontent.com)... 108.177.119.132, 2a00:1450:4013:c00::84
Connecting to doc-0g-28-docs.googleusercontent.com (doc-0g-28-

In [75]:
!pip install --quiet tensorflow-text

import tensorflow_text as text
from tensorflow.python.platform import gfile

model_file = 'hiragana_jsut.model'
model = gfile.GFile(model_file, 'rb').read()

tokenizer = text.SentencepieceTokenizer(model=model)

input_vocab_size = tokenizer.vocab_size().numpy()
print(input_vocab_size)

2000


In [76]:
if COLAB_ENVIRONMENT:
  !pip install pykakasi
else:
  !pip install --user pykakasi

In [77]:
import pykakasi
converter = pykakasi.kakasi()

In [78]:
def preprocess_text(old_text):
  text = old_text
  text = tokenizer.tokenize(text)
  text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  return text

In [79]:
def preprocess_text_no_pad(old_text):
  text = old_text
  text = tokenizer.tokenize(text)
  text = tf.shape(text)[0]
  return text

In [80]:
def preprocess_audio(old_audio, original_sample_rate):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, original_sample_rate, 24000)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)
  audio = tf.pad(audio, paddings=[[0 , 500 - tf.shape(audio)[0]], [0,0]])
  audio = tf.cast(audio, tf.float32)
  return audio

In [81]:
def preprocess_audio_no_pad(old_audio, original_sample_rate):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, original_sample_rate, 24000)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)
  audio = tf.shape(audio)[0]
  return audio

In [82]:
download_config = tfds.download.DownloadConfig(manual_dir='gs://general-304503/public_datasets/downloads/manual')

jsut_dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='gs://general-304503/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [83]:
def convert_to_kana(text):
  #kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
  new_text = text.numpy().decode('utf-8')
  kana = converter.convert(new_text)
  return ''.join([i['hira'] for i in kana])

In [84]:
jsut_dataset = jsut_dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": tf.py_function(convert_to_kana, [x["text"]], Tout=tf.string)})

In [85]:
jsut_dataset = jsut_dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"], 48000),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"], 48000),
                                 "text": tf.py_function(preprocess_text, [x["text"]], Tout=tf.int32),
                                 "text_lengths": tf.py_function(preprocess_text_no_pad, [x["text"]], Tout=tf.int32)})

In [86]:
jsut_dataset = jsut_dataset.batch(BATCH_SIZE)
jsut_dataset = jsut_dataset.prefetch(tf.data.AUTOTUNE)

## jvs dataset

In [87]:
download_config = tfds.download.DownloadConfig(manual_dir='gs://general-304503/public_datasets/downloads/manual')

jvs_dataset, info = tfds.load(
                    "jvs",
                    split="train",
                    data_dir='gs://general-304503/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [88]:
jvs_dataset = jvs_dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": tf.py_function(convert_to_kana, [x["text"]], Tout=tf.string)})

In [89]:
jvs_dataset = jvs_dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"], 48000),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"], 48000),
                                 "text": tf.py_function(preprocess_text, [x["text"]], Tout=tf.int32),
                                 "text_lengths": tf.py_function(preprocess_text_no_pad, [x["text"]], Tout=tf.int32)})

In [90]:
jvs_dataset = jvs_dataset.batch(BATCH_SIZE)
jvs_dataset = jvs_dataset.prefetch(tf.data.AUTOTUNE)



```python
ds = jvs_dataset.take(1)

for i in ds:
  data = i
  break

speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']

tokenizer.detokenize(text.numpy()[1]).numpy().decode('utf-8')
```



In [91]:
ds = jvs_dataset.take(1)
 
for i in ds:
  data = i
  break
 
speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']
 
tokenizer.detokenize(text.numpy()[1]).numpy().decode('utf-8')

'かさいですべてのこんぴゅーたーでぃすくがだめになってしまったとき、かいしゃは、もうおてあげのじょうきょうだった ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ '

In [92]:
input_vocab_size = tokenizer.vocab_size()
print(input_vocab_size)

tf.Tensor(2000, shape=(), dtype=int32)


In [139]:
data['speech'].dtype

tf.float32

In [140]:
data['speech']

<tf.Tensor: shape=(8, 500, 1025), dtype=float32, numpy=
array([[[-5.50772878e-04, -1.33253331e-03,  1.92459440e-03, ...,
          7.29341991e-08, -1.69558916e-07,  1.65047823e-07],
        [ 9.53423907e-04, -4.61390195e-03,  8.26186035e-03, ...,
          3.77185643e-08, -9.08039510e-09,  3.41096893e-08],
        [-1.93090911e-03,  1.39400014e-03, -2.83157127e-03, ...,
         -1.81491487e-07,  5.20376489e-07, -4.83356416e-07],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-1.98839419e-03,  8.59539141e-04, -3.49902222e-03, ...,
         -6.01168722e-07,  6.68514986e-08,  1.76485628e-07],
        [-6.57372118e-04, -2.24305922e-03,  6.7

In [145]:
#encoder(data['speech'])

# Gated Convolutional Network

In [146]:
d_model = 512

In [147]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [148]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [149]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='validation_loss')
instant_loss = tf.keras.metrics.Mean(name='instant_loss')

epoch_train_loss = tf.keras.metrics.Mean(name='epoch_train_loss')

In [150]:
class GatedConvolution(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, dropout_rate, padding='causal'):
    super(GatedConvolution, self).__init__()

    self.convolution = tf.keras.layers.Conv1D(
    filters=filters, kernel_size=kernel_size, padding=padding,
    )

    self.gated = tf.keras.layers.Conv1D(
    filters=filters, kernel_size=kernel_size, padding=padding,
    )

    self.multiply = tf.keras.layers.Multiply()
    
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
  def call(self, x, training):

    convolution_output = self.convolution(x)  # (batch_size, input_seq_len, d_model)

    gate_output = tf.keras.activations.sigmoid(self.gated(x))
    
    output = self.multiply([convolution_output, gate_output])

    output = self.dropout(output, training=training)
    
    return output

In [151]:
class GatedConvolutionalEncoder(tf.keras.Model):
  def __init__(self):
    super(GatedConvolutionalEncoder, self).__init__()

    self.num_convolutions = 14
  
    self.convolutions = [GatedConvolution(filters=100, kernel_size=3, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=100, kernel_size=4, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=100, kernel_size=5, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=125, kernel_size=6, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=125, kernel_size=7, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=150, kernel_size=8, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=175, kernel_size=9, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=200, kernel_size=10, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=225, kernel_size=11, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=250, kernel_size=12, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=250, kernel_size=13, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=250, kernel_size=14, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=300, kernel_size=15, padding='causal', dropout_rate=0.25),
                        GatedConvolution(filters=300, kernel_size=21, padding='causal', dropout_rate=0.25)]

    self.fc1 = tf.keras.layers.Dense(1000)
    self.final_layer = tf.keras.layers.Dense(input_vocab_size)

  def call(self, inp, training=True):
    output = inp
    for i in range(self.num_convolutions):
      output = self.convolutions[i](output, training=training)
    #output = self.convolutions[0]
    #output = self.convolutions[1]

    output = self.fc1(output)
    output = self.final_layer(output)
    return output



In [152]:
encoder = GatedConvolutionalEncoder()

In [153]:
#encoder.summary()

In [154]:
batch_counter = tf.Variable(-1, name="batch_counter", dtype=tf.int32)
epoch_counter = tf.Variable(0, name="epoch_counter", dtype=tf.int32)

ckpt = tf.train.Checkpoint(encoder=encoder,
                           optimizer=optimizer,
                          batch_counter=batch_counter,
                           epoch_counter=epoch_counter)

#ckpt = tf.train.Checkpoint(encoder=encoder,
#                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [155]:
@tf.function
def train_step(speech, text, speech_lengths, text_lengths):
  with tf.GradientTape() as tape:
    logits = encoder(speech)

    labels = text
    label_length = text_lengths
    logit_length = speech_lengths
    unique = tf.nn.ctc_unique_labels(text)


    loss = tf.nn.ctc_loss(
        labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
    )

  gradients = tape.gradient(loss, encoder.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))

  train_loss(loss)
  instant_loss(loss)
  epoch_train_loss(loss)
  return logits
    

In [156]:
@tf.function
def validation_step(speech, text, speech_lengths, text_lengths):
  logits = encoder(speech)

  labels = text
  label_length = text_lengths
  logit_length = speech_lengths
  unique = tf.nn.ctc_unique_labels(text)


  loss = tf.nn.ctc_loss(
      labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
  )


  validation_loss(loss)
  return logits

In [157]:
EPOCHS = 5
validate_period = 30
save_period = 52
print_period = 1
train_period = 30

In [158]:
batch_offset = batch_counter.numpy() + 1
epoch_offset = epoch_counter.numpy()

In [159]:
jvs_dataset.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=1873>

In [160]:
dataset = jvs_dataset.take(100)

In [161]:
list(range(1,3))

[1, 2]

In [162]:
# Resume at the batch we left off at if there was some interruption
if batch_counter != 0 or epoch_counter != 0:
    dataset = dataset.skip(batch_offset)
    print("Resuming from batch:", batch_offset, 'epoch:', epoch_offset)

for epoch in range(epoch_offset, EPOCHS):
  epoch_counter.assign(epoch)
  start = time.time()

  train_loss.reset_states()
  validation_loss.reset_states()
  instant_loss.reset_states()

  
  for (batch, data) in enumerate(dataset):
    batch += batch_offset
    batch_counter.assign(batch)
    print(batch)
    #if batch == 9:
    #  print(tokenizer.detokenize(data['text']).numpy()[0].decode('utf-8'))
    #  print(data['text'].numpy()[0])
    #  continue
    
    #texts = list([i for i in tokenizer.detokenize(tf.boolean_mask(data['text'], data['text'] != 0)).numpy()])
    #print(ids)
    
    ids = list(data['id'].numpy())
    texts = []
    for i in range(len(ids)):
        texts.append(tokenizer.detokenize(tf.boolean_mask(data['text'][i], data['text'][i] != 0)).numpy().decode('utf-8'))
    text = ''
    for i in range(len(ids)):
        text += 'id: {}'.format(ids[i]) + '\n' + 'text: {}'.format(texts[i]) + '\n'
    #print(text)

    logits = train_step(data['speech'], data['text'], data['speech_lengths'], data['text_lengths'])
    # Prevent predicting 0
    


    if (batch+1) % print_period == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {instant_loss.result():.4f}')
      instant_loss.reset_states()

    if (batch+1) % train_period == 0:
      with file_writer.as_default():
        tf.summary.scalar("train_loss", train_loss.result(), step=batch)
        
        
        
      print(f'Training Print: Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f}')
      train_loss.reset_states()

    if (batch + 1) % save_period == 0:
      ckpt_save_path = ckpt_manager.save()
      print (f'Saving checkpoint for batch {batch} at {ckpt_save_path}')

      instance_repeat = tf.repeat(tf.expand_dims(tf.range(2000), axis=0), 500, axis=0)
      batch_repeat = tf.repeat(tf.expand_dims(instance_repeat, axis=0), BATCH_SIZE, axis=0)
      logits_zero_removed = tf.where(batch_repeat != 0, logits, -np.inf)
      logits = logits_zero_removed
    
      tokenized = tf.math.argmax(logits, axis=2)
      tokenized = tf.cast(tokenized, tf.int32)
      detokenized = tokenizer.detokenize(tokenized).numpy()
    #print(detokenized.numpy()[0].decode('utf-8'))

      ids = list(data['id'].numpy())
      texts = []
      for i in range(len(ids)):
        texts.append(tokenizer.detokenize(tf.boolean_mask(data['text'][i], data['text'][i] != 0)).numpy().decode('utf-8'))
      text = ''
      for i in range(len(ids)):
        text += 'id: {}'.format(ids[i]) + '\n\n' + 'text: {}'.format(texts[i]) + '\n\n' + 'predictions: {}'.format(detokenized[i].decode('utf-8')) + '\n\n\n'
        #print(text)
      with file_writer.as_default():
        tf.summary.text("Training Text. Epoch: {}, Batch: {}".format(epoch, batch), text, step=batch)

    if (batch + 1) % validate_period == 0:
      for (val_batch, val_data) in enumerate(jsut_dataset.take(5)):
        validation_step(val_data['speech'], val_data['text'], val_data['speech_lengths'], val_data['text_lengths'])
      with file_writer.as_default():
        tf.summary.scalar("validation_loss", validation_loss.result(), step=batch)
      print(f'Validation Print: Epoch {epoch + 1} Batch {batch} Loss {validation_loss.result():.4f}')
      validation_loss.reset_states()

  
  batch_offset = 0
  print(f'Epoch {epoch + 1} Loss {epoch_train_loss.result():.4f}')
  epoch_train_loss.reset_states()

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


Resuming from batch: 0 epoch: 0
0
Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


Epoch 1 Batch 0 Loss 561.9199
1
Epoch 1 Batch 1 Loss 536.9805
2
Epoch 1 Batch 2 Loss 580.2170
3
Epoch 1 Batch 3 Loss 545.3163
4
Epoch 1 Batch 4 Loss 577.1151
5
Epoch 1 Batch 5 Loss 728.5364
6
Epoch 1 Batch 6 Loss 440.2622
7
Epoch 1 Batch 7 Loss 490.1438
8
Epoch 1 Batch 8 Loss 654.2244
9
Epoch 1 Batch 9 Loss 518.5106
10
Epoch 1 Batch 10 Loss 484.0682
11
Epoch 1 Batch 11 Loss 481.7744
12
Epoch 1 Batch 12 Loss 652.9542
13
Epoch 1 Batch 13 Loss 494.5081
14
Epoch 1 Batch 14 Loss 698.2598
15
Epoch 1 Batch 15 Loss 485.7914
16
Epoch 1 Batch 16 Loss 601.5153
17
Epoch 1 Batch 17 Loss 502.5918
18
Epoch 1 Batch 18 Loss 587.0679
19
Epoch 1 Batch 19 Loss 478.9898
20
Epoch 1 Batch 20 Loss 420.2528
21
Epoch 1 Batch 21 Loss 694.9532
22
Epoch 1 Batch 22 Loss 443.5592
23
Epoch 1 Batch 23 Loss 585.6325
24
Epoch 1 Batch 24 Loss 654.7162
25
Epoch 1 Batch 25 Loss 609.1465
26
Epoch 1 Batch 26 Loss 551.1705
27
Epoch 1 Batch 27 Loss 618.1853
28
Epoch 1 Batch 28 Loss 529.0765
29
Epoch 1 Batch 29 Loss 545.0487
Tr

KeyboardInterrupt: ignored